In [1]:
from datasets import load_dataset

ds = load_dataset('openpecha/cleaned_MT_v1.0.3', split='test')

In [ ]:
def translate(text):
    messages = [
        {"role": "system", "content": "You are a skilled translator of Tibetan texts. You answer only with the requested translation and nothing else."},
        {"role": "user", "content": f"Translate from Tibetan to English: {text}"}
        ]

    outputs = pipe(
        messages,
        max_new_tokens=256,
        )
    return outputs[0]["generated_text"][-1]['content']

In [ ]:
def get_translations(example):
    example['pred'] = translate(example['Source'])
    return example

In [ ]:
ds = ds.map(get_translations)

In [ ]:
import evaluate

# Load BLEU and CHRF metrics
bleu_metric = evaluate.load("sacrebleu")
chrf_metric = evaluate.load("chrf")
ter_metric = evaluate.load("ter")

preds = ds['pred']
labels = ds['Target']

# Compute BLEU score
bleu_result = bleu_metric.compute(predictions=preds, references=labels)
bleu_score = bleu_result["score"]

# Compute CHRF score
chrf_result = chrf_metric.compute(predictions=preds, references=labels)
chrf_score = chrf_result["score"]

# Compute TER score
ter_result = ter_metric.compute(predictions=preds, references=labels)
ter_score = ter_result["score"]

metrics = {
        "bleu": round(bleu_score, 4),
        "chrf": round(chrf_score, 4),
        "ter": round(ter_score, 4)
    }

print("Computed Metrics:", metrics)